# Getting all available Swiss supermarkets using overpass Web API

## Overpass turbo query to get all available supermarkets in Switzerland

In [2]:
!pip install pandas
!pip install folium

from pandas import json_normalize
import requests
import json

# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-1"="CH"][admin_level=2];
        node ["shop"="supermarket"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to file
with open('supermarkets.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = json_normalize(data)
df.head(5)

  Using cached folium-0.12.1.post1-py2.py3-none-any.whl (95 kB)
  Using cached branca-0.5.0-py3-none-any.whl (24 kB)


,type,id,lat,lon,tags.brand,tags.brand:wikidata,tags.brand:wikipedia,tags.name,tags.opening_hours,tags.shop,...,tags.image,tags.opening_date,tags.room,tags.zo_area,tags.addr:city:de,tags.addr:city:fr,tags.payment:discover_card,tags.trade,tags.source:addr,tags.postid
0,node,33126515,47.155616,9.037915,Spar,Q610492,en:SPAR (retailer),Spar,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00,supermarket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,36726161,47.226191,8.980329,Migros,Q680727,de:Migros,Migros,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17...",supermarket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,39768209,47.225069,8.969981,Coop,Q432564,NaN,Coop,NaN,supermarket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,39947904,47.376732,8.542161,Coop,Q432564,de:Coop (Schweiz),Coop,Mo-Sa 06:00-22:00,supermarket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,48932835,47.375020,8.522895,Migros,Q680727,de:Migros,Migros,Mo-Sa 08:00-21:00; PH off,supermarket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Plot supermarkets on map

In [3]:
import folium
import pandas as pd

# Subset of supermarkets by brand
locations = df[["lat", "lon", "tags.brand", "tags.shop"]].loc[df["tags.brand"] == 'Migros']
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                  popup=location_info["tags.brand"]).add_to(map)

# Plot map
map

          lat       lon tags.brand    tags.shop
1   47.226191  8.980329     Migros  supermarket
4   47.375020  8.522895     Migros  supermarket
7   47.491874  8.706448     Migros  supermarket
11  47.344749  8.529981     Migros  supermarket
16  47.375255  8.536107     Migros  supermarket
